In [180]:
using RigidBodyDynamics
using GeometryTypes
using AffineTransforms
using LCMGL
using Interact
using ForwardDiff
using DataFrames
using SpatialFields
import RigidBodyTreeInspector
import DrakeVisualizer: Visualizer, draw
import DataStructures: OrderedDict

In [181]:
include("arms.jl")
include("depth_sensors.jl")

DepthSensors

In [209]:
function beanbag()
    limbs = OrderedDict{RigidBody, Arms.Limb}()
    
    mechanism = Mechanism{Float64}("world")
    parent = root_body(mechanism)
    
    joint = Joint("joint1", QuaternionFloating())
    joint_to_parent = Transform3D(Float64, joint.frameBefore, parent.frame)
    body = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")))
    body_to_joint = Transform3D(Float64, body.frame, joint.frameAfter)
    attach!(mechanism, parent, joint, joint_to_parent, body, body_to_joint)
    
    surface_points = Vector{Point3D}()
    skeleton_points = [Point3D(body.frame, Vec(0.,0,0))]
    for axis = Any[[1], [2], [3], [1,2], [1,3], [2,3]]
        for s = [-1; 1]
            x = [0.; 0; 0]
            x[axis] = s
            push!(surface_points, Point3D(body.frame, Vec(x)))
        end
    end
    limbs[body] = Arms.Limb(surface_points, skeleton_points)
    
    Arms.Model(mechanism, limbs)
end

beanbag (generic function with 1 method)

In [212]:
model = beanbag()

Arms.Model(Vertex: world (root)
  Vertex: body1, Edge: joint1,DataStructures.OrderedDict{RigidBodyDynamics.RigidBody{T<:Real},Arms.Limb}(RigidBody: "body1"=>Arms.Limb(RigidBodyDynamics.Point3D[Point3D in "body1": Vec(-1.0,0.0,0.0),Point3D in "body1": Vec(1.0,0.0,0.0),Point3D in "body1": Vec(0.0,-1.0,0.0),Point3D in "body1": Vec(0.0,1.0,0.0),Point3D in "body1": Vec(0.0,0.0,-1.0),Point3D in "body1": Vec(0.0,0.0,1.0),Point3D in "body1": Vec(-1.0,-1.0,0.0),Point3D in "body1": Vec(1.0,1.0,0.0),Point3D in "body1": Vec(-1.0,0.0,-1.0),Point3D in "body1": Vec(1.0,0.0,1.0),Point3D in "body1": Vec(0.0,-1.0,-1.0),Point3D in "body1": Vec(0.0,1.0,1.0)],RigidBodyDynamics.Point3D[Point3D in "body1": Vec(0.0,0.0,0.0)])))

In [213]:
state = Arms.ModelState(model)
draw(model, state)

In [214]:
state.limb_deformations[1][1].frame

CartesianFrame3D: "body1"

In [215]:
joint_angles, deformations = Arms.zero_configuration(model)
state = Arms.ModelState(model, joint_angles, deformations)
@manipulate for x = linspace(-1, 1, 51)
    state.limb_deformations[1][1] = FreeVector3D(model.mechanism.toposortedTree[2].vertexData.frame, Vec(x, 0, 0))
    draw(model, state)
end

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"x",0.0,linspace(-1.0,1.0,51),true)

nothing

In [216]:
# Construct the sensor and draw its view rays for debugging
sensor = DepthSensors.Kinect(80,80);
camera_origin = tformtranslate([0;0;10]) * tformrotate([pi;0;0])
LCMGLClient("sensor_rays") do lcmgl
    LCMGL.color(lcmgl, 0, 1, 0)
    begin_mode(lcmgl, LCMGL.LINES)
    for ray in sensor.rays
        vertex(lcmgl, camera_origin.offset...)
        vertex(lcmgl, (camera_origin.offset + camera_origin.scalefwd * convert(Vector, ray))...)
    end
    end_mode(lcmgl)
    switch_buffer(lcmgl)
end

In [217]:
function raycast(model::Arms.Model, state::Arms.ModelState)
    draw(model, state, true)
    skin = Arms.skin(model, state)
    
    points = DepthSensors.raycast_points(skin, sensor, camera_origin)
    LCMGLClient("raycast") do lcmgl
        LCMGL.color(lcmgl, 0, 1, 0)
        point_size(lcmgl, 5)
        begin_mode(lcmgl, LCMGL.POINTS)
        for point in points
            vertex(lcmgl, convert(Vector, point)...)
        end
        end_mode(lcmgl)
        switch_buffer(lcmgl)
    end
    points
end

raycast (generic function with 10 methods)

In [218]:
joint_angles, deformations = Arms.zero_configuration(model)
joint_angles[5:7] = 2*rand(3)
for (i, limb_deformations) in enumerate(deformations)
    for (j, deformation) in enumerate(limb_deformations)
        deformations[i][j] = rand(Vec{3, Float64}) - 0.5
    end
end
state = Arms.ModelState(model, joint_angles, deformations)
skin = Arms.skin(model, state)

SpatialFields.InterpolatingSurface{3,Float64,SpatialFields.XSquaredLogX}([0.005735340484620383,0.7041256259276711,0.271796481796155,0.049741618762791553,0.058906109302351774,-0.009606576198673833,0.007407439331499264,-0.12516034806348916,0.08573999890132918,-0.13907763363167897,-0.04683459791125678,0.26576001976313113,-1.1285334784644507],13-elementArray{FixedSizeArrays.Point{3,Float64},1}:
 Point(-0.3899001554598547,1.1834354367028566,1.1751020772575198)
 Point(1.8323239556834718,1.1282878703074126,0.8184683885856787)
 Point(0.973957314721644,0.13659077667214548,1.3621845239961767)
 Point(1.1323157588315815,2.337113319646515,0.7045512757039893)
 Point(1.3664864336403832,1.48641895374533,-0.4153327386260126)
 Point(0.9327613290266719,1.2237547838634606,1.9131257535944675)
 Point(-0.28244144832183204,0.15829537784093684,0.8122134217684922)
 Point(2.3014717832124694,1.7861701872737559,0.8168087001985276)
 Point(0.2620446099608449,0.7896734950290372,-0.5772872430120326)
 Point(2.434183696

In [219]:
raycast(model, state)

448-element Array{FixedSizeArrays.Point{3,Float64},1}:
 Point(0.5154575469941564,2.2652689662426315,0.8264075153828383)  
 Point(0.6201021875970781,2.22966676479739,0.9705846936698066)    
 Point(0.7258162956979488,2.2082729075929564,1.0572227620812118)  
 Point(0.8317708910158254,2.1932181970324227,1.118189376516085)   
 Point(0.9377335451325407,2.1817245208565095,1.1647349757183143)  
 Point(1.0437045847889688,2.172668022063906,1.201410810022697)    
 Point(1.149859913015401,2.165683665764153,1.2296951503884994)    
 Point(1.2565120220980006,2.160768272239735,1.2496008736233577)   
 Point(1.364018918775055,2.1579916869418474,1.2608451286770297)   
 Point(1.4726217096341232,2.1572318142733913,1.2639223624644949)  
 Point(1.5822118165637493,2.1579233174596575,1.2611220025394037)  
 Point(1.6921110770761432,2.15892003535653,1.2570856236615366)    
 Point(1.8011000022336396,2.158704858317223,1.2579570197297567)   
 ⋮                                                                
 Point(

In [220]:
function flatten(state::Arms.ModelState)
    data = vcat(values(state.mechanism_state.q)...)
    for limb_deformation in state.limb_deformations
        for d in limb_deformation
            for x in d.v
                push!(data, x)
            end
        end
    end
    data
end

function unflatten(model::Arms.Model, state::Arms.ModelState, data)
    data_index = 1
    for joint in keys(state.mechanism_state.q)
        nq = RigidBodyDynamics.num_positions(joint)
        state.mechanism_state.q[joint][:] = data[data_index:(data_index+nq-1)]
        data_index += nq
    end
    for (i, (body, limb)) in enumerate(model.limbs)
        limb_deformation = state.limb_deformations[i]
        for (j, deformation) in enumerate(limb_deformation)
            state.limb_deformations[i][j] = RigidBodyDynamics.FreeVector3D(body.frame, Vec(data[data_index:data_index+2]))
            data_index += 3
        end
    end
    @assert data_index == length(data) + 1
    RigidBodyDynamics.setdirty!(state.mechanism_state)
    state
end

unflatten (generic function with 10 methods)

In [240]:
joint_angles, deformations = Arms.zero_configuration(model)
joint_angles[5:7] = rand(3)
for (i, limb_deformations) in enumerate(deformations)
    for (j, deformation) in enumerate(limb_deformations)
        deformations[i][j] = 0.5*(rand(Vec{3, Float64}) - 0.5)
    end
end

true_state = Arms.ModelState(model, joint_angles, deformations)
x_true = flatten(true_state)



sensed_points = raycast(model, true_state)

x_ad = []
state_ad = []

function cost{T}(x::Vector{T})
    push!(x_ad, x)
    state = Arms.ModelState(model, T, T)
    unflatten(model, state, x)
    push!(state_ad, state)
    draw(model, state, false)
    skin = Arms.skin(model, state)
    raycast_cost = sum(point -> evaluate(skin, point)^2, sensed_points)
    deformation_cost = zero(T)
    for ld in state.limb_deformations
        for d in ld
            deformation_cost += sum(d.v .^2)
        end
    end
    return raycast_cost + 100*deformation_cost
end

# Use autodiff to prepare a function that will evaluate the gradient
# of that cost:
g_cost = ForwardDiff.gradient(cost)

trials = []

function test_gradient_descent()
    for i = 1:1
        xs = []

        # We'll initialize our estimate with a uniformly random pair
        # of joint angles
        x_estimated = flatten(Arms.ModelState(model))

        # Now we use gradient descent to recover the joint angles
        for i = 1:200
            push!(xs, x_estimated)

            # Evaluate the gradient of the point values w.r.t. the
            # joint angles:
            gi = g_cost(x_estimated)

            if any(isnan, gi)
                @show x_estimated
                error("NaN in gradient")
            end

            # Descend the gradient
            x_estimated -= 0.0005 * gi
        end
        push!(trials, DataFrame(x=xs))
    end
end

test_gradient_descent (generic function with 1 method)

In [241]:
test_gradient_descent()
state = unflatten(model, Arms.ModelState(model), trials[1][:x][end])
draw(model, state)